In [1]:
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

In [ ]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-25 17:31:28.551579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 17:31:28.551645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 17:31:28.553126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
# T5_model="/kaggle/input/nlpprojt5/T5_overall"#phrase
T5_model="/kaggle/input/t5-retriver/retriever_passage/T5_overall_passage"
# T5_model="/kaggle/input/nlpprojt5/T5_overall_multi"
sep='[SEP]'

In [ ]:
tag="passage"

In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [6]:
def remove_escape(l):
    return l.replace("\n",". ")

In [7]:
def get_label(x):
    return " ".join(x)

In [8]:
train_data=pd.read_json("/kaggle/input/retriever/final_data_train.json")
val_data=pd.read_json("/kaggle/input/retriever/final_data_validation.json")
test_data=pd.read_json("/kaggle/input/retriever/final_data_test.json")

train_data = train_data.replace({None: ''})
val_data = val_data.replace({None: ''})
test_data = test_data.replace({None: ''})

train_real=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True)
val_real=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True)
test_real=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True)

train_labels=train_real['spoiler'].apply(get_label)
val_labels=val_real['spoiler'].apply(get_label)
test_labels=test_real['spoiler'].apply(get_label)

train_tags=train_real['tags'].apply(get_label)
val_tags=val_real['tags'].apply(get_label)
test_tags=test_real['tags'].apply(get_label)
# train_type=train_real['spoiler']
train_data['exctractedParagraph']=train_data['exctractedParagraph'].apply(remove_escape)
val_data['exctractedParagraph']=val_data['exctractedParagraph'].apply(remove_escape)
test_data['exctractedParagraph']=test_data['exctractedParagraph'].apply(remove_escape)

train_data['context']=train_data['targetTitle'] + ' ' + train_data['targetDescription']+" "+train_data['exctractedParagraph']
val_data['context']=val_data['targetTitle'] + ' ' + val_data['targetDescription']+" "+val_data['exctractedParagraph']
test_data['context']=test_data['targetTitle'] + ' ' + test_data['targetDescription']+" "+test_data['exctractedParagraph']

train_data['question']=train_data['postText']+"?"
val_data['question']=val_data['postText']+'?'
test_data['question']=test_data['postText']+"?"

train_data['spoiler']=train_labels
val_data['spoiler']=val_labels
test_data['spoiler']=test_labels

train_data['tags']=train_tags
val_data['tags']=val_tags
test_data['tags']=test_tags

train_data=train_data[train_data['tags']==tag]
val_data=val_data[val_data['tags']==tag]
test_data=test_data[test_data['tags']==tag]

train_data=train_data[['question','context','spoiler']]

val_data=val_data[['question','context','spoiler']]
test_data=test_data[['question','context','spoiler']]


In [9]:
train_data

,question,context,spoiler
0,"Wes Welker Wanted Dinner With Tom Brady, But P...","Wes Welker Wanted Dinner With Tom Brady, But P...",how about that morning we go throw?
5,What happens if your new AirPods get lost or s...,Here's what happens if your Apple AirPods get ...,"Apple says that if AirPods are lost or stolen,..."
6,The Reason Why Gabor Kiraly Wears THOSE Tracki...,The Reason Why Gabor Kiraly Wears THOSE Tracki...,"""The more good games I had in them, the more I..."
7,You’ll Never Believe What This Family Saw in t...,You’ll Never Believe What This Family Saw in t...,rainbow colours in the sky and a halo spanning...
8,Should you drink Red Wine??,Should I Drink Red Wine? Is red wine healthy a...,Red wine is clearly the drink of choice if you...
...,...,...,...
3189,Women Reveal What They Think The Sexiest Part ...,Women Reveal What They Think The Sexiest Part ...,almost every other part of a man is considered...
3193,Here's another huge reason to eat a plant-base...,Here's Another Huge Reason To Eat A Plant-Base...,may cut your risk for colorectal cancer -- the...
3194,If You See A Purple Butterfly Sticker At The H...,If You See A Purple Butterfly Sticker At The H...,one of her unborn twins had a condition called...
3195,Has Facebook's video explosion completely shak...,"Facebook Video Surging, But YouTube Still Offe...",it hasn’t necessarily taken the wind out of Yo...


In [10]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        if c[i]==None:
            c[i]=""
        a.append(str(data['spoiler'][i]))
    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs

In [11]:
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [12]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_dataset = Dataset(pa.Table.from_pandas(train_data)).remove_columns(["__index_level_0__"])

val_dataset = Dataset(pa.Table.from_pandas(val_data)).remove_columns(["__index_level_0__"])
test_dataset = Dataset(pa.Table.from_pandas(test_data)).remove_columns(["__index_level_0__"])
train_dataset,val_dataset,test_dataset

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 1274
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 322
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 403
 }))

In [13]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

Map:   0%|          | 0/403 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1274
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 322
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 403
 }))

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [16]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]



In [20]:
# results=trainer.evaluate()

In [21]:
# calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [22]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)
wandb.init()

wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [23]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.027839987841166734, 'precisions': [0.19038534910339566, 0.06178962595577599, 0.04121621621621622, 0.032665181885671864], 'brevity_penalty': 0.4413096950102145, 'length_ratio': 0.5500524658971668, 'translation_length': 5242, 'reference_length': 9530}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 10.15523624420166, 'eval_blue': 0.027839987841166734, 'eval_precisions_1': 0.19038534910339566, 'eval_precisions_2': 0.06178962595577599, 'eval_precisions_3': 0.04121621621621622, 'eval_precisions_4': 0.032665181885671864, 'eval_bp': 0.4413096950102145, 'eval_meteor': 0.11294342066693827, 'eval_bertscore_f1': 0.8436320512821301, 'eval_bertscore_p': 0.8464283518696541, 'eval_bertscore_r': 0.8412937940498144, 'eval_runtime': 47.8031, 'eval_samples_per_second': 8.43, 'eval_steps_per_second': 1.067}


In [24]:
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

[0.08401890034723276,
 0.04786499456032086,
 0.03556289830856601,
 0.027839987841166747]